In [4]:
import os
os.chdir('/Users/omii/Desktop/Brain-Tumor-Classification/')

In [5]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow_hub as hub 
import os
import mlflow.keras
import mlflow


# os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = './src/keys/tensile-topic-424308-d9-7418db5a1c90'

keyfile_path = 'src/keys/tensile-topic-424308-d9-7418db5a1c90.json'  #change as per your keyfile path

# Checking if file exists
if not os.path.exists(keyfile_path):
    raise FileNotFoundError(f"The file '{keyfile_path}' does not exist. Please check the path.")

# Set the environment variable to point to the service account key file
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = keyfile_path
os.environ['MLFLOW_GCS_BUCKET'] = 'ml-flow-remote-tracker-bucket'

mlflow.set_tracking_uri("http://35.231.231.140:5000/")
mlflow.set_experiment("Brain-Tumor-Classification")



physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    try:
        tf.config.experimental.set_memory_growth(physical_devices[0], True)
        print("TensorFlow with Metal enabled with {} GPUs".format(len(physical_devices)))
    except RuntimeError as e:
        print(e)


def preprocessing_for_training():
    path = './data/Training/'
    train_datagen = ImageDataGenerator(
        rescale=1.0/255,
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        fill_mode='nearest'
    )
    train_generator = train_datagen.flow_from_directory(
        path,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical',
        shuffle=True,
        seed=42
    )
    return train_generator

def preprocessing_for_testing_inference(batchSize):
    path = './data/Testing/'
    test_val_datagen = ImageDataGenerator(rescale=1.0/255)
    test_generator = test_val_datagen.flow_from_directory(
        path,
        target_size=(224, 224),
        batch_size=batchSize,
        class_mode='categorical',
        shuffle=False
    )
    return test_generator


model = models.Sequential([
    layers.InputLayer(shape=(224, 224, 3)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(4, activation='sigmoid')
])
model.summary()

train_dataset = preprocessing_for_training()
validation_dataset = preprocessing_for_testing_inference(batchSize=32)


with mlflow.start_run() as run:
    # Log parameters
    mlflow.log_param("batch_size", 32)
    mlflow.log_param("epochs", 10)
    mlflow.log_param("optimizer", "adam")
    mlflow.log_param("loss_function", "binary_crossentropy")
    mlflow.log_param("metrics", "accuracy")

    model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

    history = model.fit(
        train_dataset,
        validation_data=validation_dataset,
        epochs=10
    )

    mlflow.keras.log_model(model, "model")

# Log metrics
for epoch in range(10):
    mlflow.log_metrics({
        "train_loss": history.history['loss'][epoch],
        "train_accuracy": history.history['accuracy'][epoch],
        "val_loss": history.history['val_loss'][epoch],
        "val_accuracy": history.history['val_accuracy'][epoch]
    }, step=epoch)


mlflow.log_artifacts('./data/Training/', artifact_path="training_data")
mlflow.log_artifacts('./data/Testing/', artifact_path="testing_data")

TensorFlow with Metal enabled with 1 GPUs


2024-06-16 13:35:15.735402: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2 Pro
2024-06-16 13:35:15.735448: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-06-16 13:35:15.735470: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-06-16 13:35:15.735801: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-06-16 13:35:15.735823: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,169,476 (42.61 MB)

...

 Non-trainable params: 0 (0.00 B)

Found 5712 images belonging to 4 classes.
Found 1311 images belonging to 4 classes.
Epoch 1/10


2024-06-16 13:35:17.688468: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
/Users/omii/anaconda3/envs/tensorflow/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


179/179 ━━━━━━━━━━━━━━━━━━━━ 35s 167ms/step - accuracy: 0.5437 - loss: 0.4541 - val_accuracy: 0.6964 - val_loss: 0.4264
Epoch 2/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 30s 158ms/step - accuracy: 0.7573 - loss: 0.2853 - val_accuracy: 0.7246 - val_loss: 0.3342
Epoch 3/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 30s 158ms/step - accuracy: 0.8029 - loss: 0.2340 - val_accuracy: 0.7948 - val_loss: 0.2533
Epoch 4/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 30s 161ms/step - accuracy: 0.8252 - loss: 0.2109 - val_accuracy: 0.8055 - val_loss: 0.2370
Epoch 5/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 30s 163ms/step - accuracy: 0.8479 - loss: 0.1889 - val_accuracy: 0.8284 - val_loss: 0.2166
Epoch 6/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 30s 162ms/step - accuracy: 0.8554 - loss: 0.1829 - val_accuracy: 0.8101 - val_loss: 0.2240
Epoch 7/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 30s 164ms/step - accuracy: 0.8624 - loss: 0.1763 - val_accuracy: 0.8253 - val_loss: 0.2037
Epoch 8/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 30s 163ms/step - accuracy: 0.8665 - loss: 0.1638 - val

2024/06/16 13:40:23 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
/Users/omii/anaconda3/envs/tensorflow/lib/python3.10/site-packages/_distutils_hack/__init__.py:26: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
